Create a first-pass of glaciers that we believe should be included in the study.

1- larger than 2 km2

2- contain at least 1 km2 debris cover from herreid et al 2020

In [1]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import pandas as pd
from datetime import timedelta
from datetime import datetime

In [2]:
# define folder and file paths
folder_HMA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"HMA2")
folder_debris = os.path.join(folder_HMA, 'Datasets', 'debris cover')
folder_rgi = os.path.join(folder_HMA, 'Datasets', 'RGI')
folder_lakes = os.path.join(folder_HMA, 'Datasets', 'lake inventories')

In [3]:
# open csvs that allow us to link rgiids between v6 and v7
link_13 = pd.read_csv(os.path.join(folder_rgi, 'links', 'RGI2000-v7.0-G-13_central_asia-rgi6_links.csv'))
link_14 = pd.read_csv(os.path.join(folder_rgi, 'links', 'RGI2000-v7.0-G-14_south_asia_west-rgi6_links.csv'))
link_15 = pd.read_csv(os.path.join(folder_rgi, 'links', 'RGI2000-v7.0-G-15_south_asia_east-rgi6_links.csv'))
# link_15.head()

In [4]:
##### open RGI v7 and herreid2020 data

# herreid debris cover, herreid glacier extents, rgi v7, and then fix 3d geometry
h_dc_15 = gpd.read_file(os.path.join(folder_debris, 'Herreid2020', '15SouthAsiaEast', '15SouthAsiaEast_minGl1km2_debrisCover.shp')).to_crs("ESRI:102025")
h_g_15 = gpd.read_file(os.path.join(folder_debris, 'Herreid2020', '15SouthAsiaEast', '15SouthAsiaEast_minGl1km2.shp')).to_crs("ESRI:102025")
rgi_15 = gpd.read_file(os.path.join(folder_rgi, 'v7', 'G', 'RGI2000-v7.0-G-15_south_asia_east', 'RGI2000-v7.0-G-15_south_asia_east.shp')).to_crs("ESRI:102025")
rgi_15['geometry'] = gpd.GeoSeries.from_wkb(rgi_15.to_wkb(output_dimension=2).geometry)

h_dc_13 = gpd.read_file(os.path.join(folder_debris, 'Herreid2020', '13CentralAsia', '13CentralAsia_minGl1km2_debrisCover.shp')).to_crs("ESRI:102025")
h_g_13 = gpd.read_file(os.path.join(folder_debris, 'Herreid2020', '13CentralAsia', '13CentralAsia_minGl1km2.shp')).to_crs("ESRI:102025")
rgi_13 = gpd.read_file(os.path.join(folder_rgi, 'v7', 'G', 'RGI2000-v7.0-G-13_central_asia', 'RGI2000-v7.0-G-13_central_asia.shp')).to_crs("ESRI:102025")
rgi_13['geometry'] = gpd.GeoSeries.from_wkb(rgi_13.to_wkb(output_dimension=2).geometry)

h_dc_14 = gpd.read_file(os.path.join(folder_debris, 'Herreid2020', '14SouthAsiaWest', '14SouthAsiaWest_minGl1km2_debrisCover.shp')).to_crs("ESRI:102025")
h_g_14 = gpd.read_file(os.path.join(folder_debris, 'Herreid2020', '14SouthAsiaWest', '14SouthAsiaWest_minGl1km2.shp')).to_crs("ESRI:102025")
rgi_14 = gpd.read_file(os.path.join(folder_rgi, 'v7', 'G', 'RGI2000-v7.0-G-14_south_asia_west', 'RGI2000-v7.0-G-14_south_asia_west.shp')).to_crs("ESRI:102025")
rgi_14['geometry'] = gpd.GeoSeries.from_wkb(rgi_14.to_wkb(output_dimension=2).geometry)

# open wang et al 2020 lake dataset
# lakes_1990 = gpd.read_file(os.path.join(folder_lakes, 'wang_2020', 'High_Asia_glacial_lake_1990.shp')).to_crs("ESRI:102025")
# lakes_2018 = gpd.read_file(os.path.join(folder_lakes, 'wang_2020', 'High_Asia_glacial_lake_2018.shp')).to_crs("ESRI:102025")

In [5]:
# concat everything into single dfs
h_dc_all = gpd.GeoDataFrame(pd.concat([h_dc_13, h_dc_14, h_dc_15]))
h_g_all = gpd.GeoDataFrame(pd.concat([h_g_13, h_g_14, h_g_15]))
rgi_all = gpd.GeoDataFrame(pd.concat([rgi_13, rgi_14, rgi_15]))
# lakes_all = gpd.GeoDataFrame(pd.concat([lakes_1990, lakes_2018]))
rgi_all.shape

(131762, 29)

In [6]:
# remove any glaciers that are less than 2 km2
rgi_all_2km = rgi_all[rgi_all['area_km2']>=2]
rgi_all_2km.shape

(8011, 29)

In [7]:
# remove lake that are not glacier-fed
# lakes_all_glacier_fed = lakes_all[lakes_all['GL_Type']=='Glacier-Fed Lake'].copy()
# lakes_all_glacier_fed.shape

In [8]:
# # Calculate the area of each resulting polygon
# intersected['area_deb'] = intersected['geometry'].area / (1000*1000)

# # aggregate intersected area within each rgiid
# total_areas = intersected[['rgi_id','area_deb']].groupby('rgi_id').sum() 

# # merge the debris intersection area back to the rgi gdf
# rgi_all_2km_debris = pd.merge(rgi_all_2km, total_areas, on=['rgi_id'], how='left')

# # fill na with 0s
# rgi_all_2km_debris['area_deb'] = rgi_all_2km_debris['area_deb'].fillna(0)

In [9]:
# # Perform overlay to get the intersection between RGI outlines and herreid glacier outlines
# # (to see what glaciers were not included by herreid)
# intersected = gpd.overlay(rgi_all_2km, h_g_union_gdf, how='intersection')

# # Calculate the area of each resulting polygon
# intersected['area_g'] = intersected['geometry'].area / (1000*1000)

# # aggregate intersected area within each rgiid
# total_areas = intersected[['rgi_id','area_g']].groupby('rgi_id').sum().rename({'area_g':'area_h'}, axis=1) 

# # merge the herreid glacier area back to the rgi gdf, fill nan with 0
# rgi_all_2km_glaciers = pd.merge(rgi_all_2km, total_areas, on=['rgi_id'], how='left')
# rgi_all_2km_glaciers['area_h'] = rgi_all_2km_glaciers['area_h'].fillna(0)

# # add column calculating area of rgiv7
# rgi_all_2km_glaciers['area_rgi7'] = rgi_all_2km_glaciers['geometry'].area / (1000*1000)
# rgi_all_2km_glaciers['area_rgi7b'] = rgi_all_2km_glaciers['area_km2']

# # add column to hold difference between the two
# rgi_all_2km_glaciers['area_missed'] = rgi_all_2km_glaciers['area_rgi7'] - rgi_all_2km_glaciers['area_h'] 
# rgi_all_2km_glaciers.head()

In [10]:
### calculate herreid-defined debris cover within each rgi7 outline

# get the geometric intersection of each rgi outline with the debris cover
intersected = gpd.overlay(rgi_all_2km, h_dc_all, how='intersection')
intersected2 = intersected.dissolve(by='rgi_id').reset_index() # dissolve because sometime these overlap

# Calculate the area of each resulting polygon
intersected2['area_dc'] = intersected2['geometry'].area / (1000*1000)

# aggregate intersected area within each rgiid
total_areas = intersected2[['rgi_id','area_dc']].groupby('rgi_id').sum()

# merge the debris intersection area back to the rgi gdf
rgi_all_2km_debris = pd.merge(rgi_all_2km, total_areas, on=['rgi_id'], how='left')

# fill na with 0s
rgi_all_2km_debris['area_dc'] = rgi_all_2km_debris['area_dc'].fillna(0)

In [11]:
### calculate the parts of each rgi7 outline that herreid did not consider

# get the geometric intersection of each rgi outline with the debris cover
intersected = gpd.overlay(rgi_all_2km, h_g_all, how='intersection')
intersected2 = intersected.dissolve(by='rgi_id').reset_index() # dissolve because sometime these overlap

# Calculate the area of each resulting polygon
intersected2['area_h'] = intersected2['geometry'].area / (1000*1000)

# aggregate intersected area within each rgiid
total_areas = intersected2[['rgi_id','area_h']].groupby('rgi_id').sum()

# merge the debris intersection area back to the rgi gdf
rgi_all_2km_missing = pd.merge(rgi_all_2km, total_areas, on=['rgi_id'], how='left')

# fill na with 0s
rgi_all_2km_missing['area_h'] = rgi_all_2km_missing['area_h'].fillna(0)

# add column calculating area of rgiv7
rgi_all_2km_missing['area_rgi7'] = rgi_all_2km_missing['geometry'].area / (1000*1000)
# rgi_all_2km_glaciers['area_rgi7b'] = rgi_all_2km_glaciers['area_km2']

# add column to hold difference between the two
rgi_all_2km_missing['area_miss'] = rgi_all_2km_missing['area_rgi7'] - rgi_all_2km_missing['area_h'] 
rgi_all_2km_missing['area_miss'] = rgi_all_2km_missing['area_miss'].round(2)
# rgi_all_2km_missing.head()

In [12]:
# merge everything back together
rgi_all_merged = pd.merge(rgi_all_2km, rgi_all_2km_debris[['rgi_id','area_dc']], on=['rgi_id'], how='left')
rgi_all_merged = pd.merge(rgi_all_merged, rgi_all_2km_missing[['rgi_id','area_h','area_miss']], on=['rgi_id'], how='left')
# rgi_all_merged['area_dc'] = rgi_all_2km_debris['area_dc'] 
# rgi_all_merged['area_miss'] = rgi_all_2km_missing['area_missed'] 
rgi_all_merged['dc_plus_m'] = rgi_all_merged['area_dc'] + rgi_all_merged['area_miss']
rgi_all_merged.shape

(8011, 33)

In [13]:
# filter to only those that has 1 km2 of debris, or 2km2 of debris+missing areas
rgi_out = rgi_all_merged [ (rgi_all_merged['dc_plus_m']>2) | 
                           (rgi_all_merged['area_dc']>1) ]
rgi_out.shape

(1577, 33)

In [14]:
# save
out_path = os.path.join(folder_HMA, "Datasets", 'AOI definition', 'prospective glaciers', "prospective_glaciers.shp")
rgi_out.to_file(out_path)

In [16]:
rgi_out.sort_values('lmax_m').head()

,rgi_id,o1region,o2region,glims_id,anlys_id,subm_id,src_date,cenlon,cenlat,utm_zone,...,slope_deg,aspect_deg,aspect_sec,dem_source,lmax_m,geometry,area_dc,area_h,area_miss,dc_plus_m
1395,RGI2000-v7.0-G-13-29346,13,13-03,G078253E41399N,868454,752,2002-07-10T00:00:00,78.253172,41.398720,44,...,28.562437,10.775922,1,COPDEM30,1745,"POLYGON ((-1265223.807 1485178.882, -1265223.6...",0.0,0.000000,2.04,2.04
7570,RGI2000-v7.0-G-15-10442,15,15-02,G091594E27979N,917775,752,1995-09-02T00:00:00,91.595747,27.981801,46,...,33.502163,217.812546,6,COPDEM30,1770,"POLYGON ((-315238.062 -232538.541, -315231.580...",0.0,0.000000,2.11,2.11
7701,RGI2000-v7.0-G-15-12195,15,15-03,G095893E30294N,930022,752,2005-09-08T00:00:00,95.890191,30.294003,46,...,16.212187,47.053852,2,COPDEM30,1837,"POLYGON ((79481.499 34509.464, 79479.346 34509...",0.0,0.025632,2.09,2.09
7047,RGI2000-v7.0-G-15-05304,15,15-01,G085141E28496N,900610,752,1998-06-18T00:00:00,85.142983,28.494304,45,...,35.666485,219.204285,6,COPDEM30,1864,"POLYGON ((-902407.637 -132225.499, -902366.299...",0.0,0.000000,2.18,2.18
7367,RGI2000-v7.0-G-15-08549,15,15-02,G086554E28049N,905019,752,1999-04-27T00:00:00,86.559606,28.046265,45,...,35.388863,117.352560,4,COPDEM30,2037,"POLYGON ((-779466.070 -197249.293, -779428.536...",0.0,0.011847,2.02,2.02
